

### Why this structure
- The src layout keeps importable code isolated from the repository root, reducing path issues and improving packaging hygiene.[1]
- SQLite via the sqlite3 module provides an embedded, zero-config relational database with DB-API 2.0 compliance and robust SQL features.[3]
- The argparse module enables a discoverable, script-friendly CLI with subcommands for products, sales, and reports without extra dependencies.[4]

### Project tree
- The following structure is ready to run and test, including services, models, utilities, CLI, data storage, tests, and environment files.[1]

```
inventory_pos_system/
│
├── src/
│   ├── __init__.py
│   ├── main.py
│   ├── models/
│   │   ├── __init__.py
│   │   ├── database.py
│   │   ├── product.py
│   │   ├── sale.py
│   │   └── customer.py
│   ├── utils/
│   │   ├── __init__.py
│   │   ├── validators.py
│   │   ├── helpers.py
│   │   └── logger.py
│   ├── services/
│   │   ├── __init__.py
│   │   ├── inventory_service.py
│   │   ├── sales_service.py
│   │   └── report_service.py
│   └── cli/
│       ├── __init__.py
│       ├── menu.py
│       └── interface.py
│
├── data/
│   └── inventory.db
│
├── tests/
│   ├── __init__.py
│   ├── test_models.py
│   ├── test_services.py
│   └── test_utils.py
│
├── requirements.txt
├── README.md
└── .gitignore
```

### Setup and run
- Use a virtual environment, install requirements, initialize the schema via first run, and execute CLI subcommands with argparse for operations and reports.[4]
- SQLite needs no separate server process; the default database file will be created on first connection in data/inventory.db per sqlite3 behavior.[3]

### src/__init__.py
- Package initializer with a simple version constant for traceability and imports.[1]

```python
# src/__init__.py
__all__ = ["models", "services", "utils", "cli"]
__version__ = "0.1.0"
```

### src/main.py
- Entry point that runs either interactive menu or argparse CLI, enabling both guided and scripted usage flows.[4]

```python
# src/main.py
from src.cli.menu import run_menu
from src.cli.interface import cli_main
import sys

def main() -> None:
    # If no args, open interactive menu; otherwise, use CLI
    if len(sys.argv) == 1:
        run_menu()
    else:
        cli_main()

if __name__ == "__main__":
    main()
```

### src/models/database.py
- Centralized database connector with schema initialization, foreign keys enabled, and convenient context management around sqlite3 connections.[3]

```python
# src/models/database.py
from __future__ import annotations

import sqlite3
from pathlib import Path
from contextlib import contextmanager
from typing import Iterator, Optional
import logging
import time

from src.utils.logger import get_logger

LOGGER = get_logger(__name__)

class Database:
    def __init__(self, db_path: Optional[Path] = None) -> None:
        if db_path is None:
            # project_root / data / inventory.db
            project_root = Path(__file__).resolve().parent.parent.parent
            db_path = project_root / "data" / "inventory.db"
        self.db_path = Path(db_path)
        self.db_path.parent.mkdir(parents=True, exist_ok=True)

    def connect(self) -> sqlite3.Connection:
        conn = sqlite3.connect(
            self.db_path,
            timeout=10.0,
            isolation_level=None,  # autocommit disabled; we'll use BEGIN...COMMIT explicitly
            detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
            check_same_thread=False,
        )
        conn.row_factory = sqlite3.Row
        conn.execute("PRAGMA foreign_keys = ON;")
        return conn

    @contextmanager
    def get_connection(self) -> Iterator[sqlite3.Connection]:
        conn = self.connect()
        try:
            yield conn
        except Exception:
            LOGGER.exception("Database error, rolling back")
            conn.rollback()
            raise
        finally:
            conn.close()

    def initialize_schema(self) -> None:
        schema = """
        BEGIN;
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sku TEXT UNIQUE NOT NULL,
            name TEXT NOT NULL,
            price REAL NOT NULL CHECK(price >= 0),
            stock INTEGER NOT NULL DEFAULT 0 CHECK(stock >= 0),
            created_at TEXT NOT NULL,
            updated_at TEXT NOT NULL
        );

        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE,
            phone TEXT,
            created_at TEXT NOT NULL
        );

        CREATE TABLE IF NOT EXISTS sales (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER,
            total REAL NOT NULL DEFAULT 0 CHECK(total >= 0),
            created_at TEXT NOT NULL,
            FOREIGN KEY(customer_id) REFERENCES customers(id) ON DELETE SET NULL
        );

        CREATE TABLE IF NOT EXISTS sale_items (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sale_id INTEGER NOT NULL,
            product_id INTEGER NOT NULL,
            quantity INTEGER NOT NULL CHECK(quantity > 0),
            unit_price REAL NOT NULL CHECK(unit_price >= 0),
            line_total REAL NOT NULL CHECK(line_total >= 0),
            FOREIGN KEY(sale_id) REFERENCES sales(id) ON DELETE CASCADE,
            FOREIGN KEY(product_id) REFERENCES products(id) ON DELETE RESTRICT
        );
        COMMIT;
        """
        with self.get_connection() as conn:
            LOGGER.info("Initializing database schema at %s", self.db_path)
            conn.executescript(schema)
            time.sleep(0.01)  # ensure file flush on some FS
```

### src/models/product.py
- Product dataclass for typed transfers and convenience converters from sqlite rows.[3]

```python
# src/models/product.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
import sqlite3

@dataclass(slots=True)
class Product:
    id: Optional[int]
    sku: str
    name: str
    price: float
    stock: int
    created_at: str
    updated_at: str

    @classmethod
    def from_row(cls, row: sqlite3.Row) -> "Product":
        return cls(
            id=row["id"],
            sku=row["sku"],
            name=row["name"],
            price=float(row["price"]),
            stock=int(row["stock"]),
            created_at=row["created_at"],
            updated_at=row["updated_at"],
        )
```

### src/models/customer.py
- Customer dataclass representing minimal customer identity with optional contact fields.[3]

```python
# src/models/customer.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
import sqlite3

@dataclass(slots=True)
class Customer:
    id: Optional[int]
    name: str
    email: Optional[str]
    phone: Optional[str]
    created_at: str

    @classmethod
    def from_row(cls, row: sqlite3.Row) -> "Customer":
        return cls(
            id=row["id"],
            name=row["name"],
            email=row["email"],
            phone=row["phone"],
            created_at=row["created_at"],
        )
```

### src/models/sale.py
- Sale and SaleItem dataclasses for clean separation of headers and line items in POS transactions.[3]

```python
# src/models/sale.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
import sqlite3

@dataclass(slots=True)
class Sale:
    id: Optional[int]
    customer_id: Optional[int]
    total: float
    created_at: str

    @classmethod
    def from_row(cls, row: sqlite3.Row) -> "Sale":
        return cls(
            id=row["id"],
            customer_id=row["customer_id"],
            total=float(row["total"]),
            created_at=row["created_at"],
        )

@dataclass(slots=True)
class SaleItem:
    id: Optional[int]
    sale_id: int
    product_id: int
    quantity: int
    unit_price: float
    line_total: float
```

### src/utils/validators.py
- Input validators for SKU, names, prices, quantities, and email to keep data consistent before persistence.[3]

```python
# src/utils/validators.py
from __future__ import annotations
import re

SKU_RE = re.compile(r"^[A-Z0-9\-]{3,32}$")
EMAIL_RE = re.compile(r"^[^@\s]+@[^@\s]+\.[^@\s]+$")

def validate_sku(sku: str) -> str:
    sku = sku.strip().upper()
    if not SKU_RE.match(sku):
        raise ValueError("SKU must be 3-32 chars, A-Z, 0-9, hyphen")
    return sku

def validate_name(name: str) -> str:
    name = name.strip()
    if not name or len(name) > 128:
        raise ValueError("Name must be 1-128 characters")
    return name

def validate_price(price: float) -> float:
    try:
        p = float(price)
    except Exception as e:
        raise ValueError("Price must be numeric") from e
    if p < 0:
        raise ValueError("Price must be >= 0")
    return round(p, 2)

def validate_quantity(qty: int) -> int:
    try:
        q = int(qty)
    except Exception as e:
        raise ValueError("Quantity must be integer") from e
    if q <= 0:
        raise ValueError("Quantity must be > 0")
    return q

def validate_email(email: str | None) -> str | None:
    if email is None:
        return None
    email = email.strip()
    if email and not EMAIL_RE.match(email):
        raise ValueError("Invalid email format")
    return email
```

### src/utils/helpers.py
- Helpers for timestamps and currency formatting to centralize formatting behavior.[3]

```python
# src/utils/helpers.py
from __future__ import annotations
from datetime import datetime

def now_iso() -> str:
    return datetime.utcnow().isoformat(timespec="seconds") + "Z"

def money(value: float) -> str:
    return f"{value:.2f}"
```

### src/utils/logger.py
- Standard logging configuration for consistent diagnostics across modules without extra dependencies.[2]

```python
# src/utils/logger.py
from __future__ import annotations
import logging
from logging import Logger

def get_logger(name: str) -> Logger:
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(logging.INFO)
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)
        fmt = logging.Formatter(
            "%(asctime)s | %(levelname)s | %(name)s | %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
        )
        ch.setFormatter(fmt)
        logger.addHandler(ch)
        logger.propagate = False
    return logger
```

### src/services/inventory_service.py
- Inventory service encapsulates product CRUD and stock adjustments with transactional safety and validation.[3]

```python
# src/services/inventory_service.py
from __future__ import annotations

from typing import Iterable, List, Optional
import sqlite3

from src.models.database import Database
from src.models.product import Product
from src.utils.helpers import now_iso
from src.utils.validators import (
    validate_name, validate_price, validate_quantity, validate_sku
)

class InventoryService:
    def __init__(self, db: Database) -> None:
        self.db = db

    def add_product(self, sku: str, name: str, price: float, stock: int = 0) -> int:
        sku = validate_sku(sku)
        name = validate_name(name)
        price = validate_price(price)
        stock = int(stock)
        if stock < 0:
            raise ValueError("Stock must be >= 0")
        ts = now_iso()
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            cur.execute(
                """
                INSERT INTO products (sku, name, price, stock, created_at, updated_at)
                VALUES (?, ?, ?, ?, ?, ?)
                """,
                (sku, name, price, stock, ts, ts),
            )
            pid = cur.lastrowid
            cur.execute("COMMIT;")
            return int(pid)

    def update_product(self, sku: str, name: Optional[str] = None, price: Optional[float] = None) -> None:
        sku = validate_sku(sku)
        fields = []
        params = []
        if name is not None:
            fields.append("name = ?")
            params.append(validate_name(name))
        if price is not None:
            fields.append("price = ?")
            params.append(validate_price(price))
        if not fields:
            return
        params.extend([now_iso(), sku])
        q = f"UPDATE products SET {', '.join(fields)}, updated_at = ? WHERE sku = ?"
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            cur.execute(q, params)
            if cur.rowcount == 0:
                cur.execute("ROLLBACK;")
                raise LookupError("Product not found")
            cur.execute("COMMIT;")

    def remove_product(self, sku: str) -> None:
        sku = validate_sku(sku)
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            cur.execute("DELETE FROM products WHERE sku = ?", (sku,))
            if cur.rowcount == 0:
                cur.execute("ROLLBACK;")
                raise LookupError("Product not found")
            cur.execute("COMMIT;")

    def get_product_by_sku(self, sku: str) -> Product:
        sku = validate_sku(sku)
        with self.db.get_connection() as conn:
            row = conn.execute("SELECT * FROM products WHERE sku = ?", (sku,)).fetchone()
            if not row:
                raise LookupError("Product not found")
            return Product.from_row(row)

    def list_products(self) -> List[Product]:
        with self.db.get_connection() as conn:
            rows = conn.execute("SELECT * FROM products ORDER BY name ASC").fetchall()
            return [Product.from_row(r) for r in rows]

    def adjust_stock(self, sku: str, delta: int) -> None:
        sku = validate_sku(sku)
        delta = int(delta)
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            row = cur.execute("SELECT id, stock FROM products WHERE sku = ?", (sku,)).fetchone()
            if not row:
                cur.execute("ROLLBACK;")
                raise LookupError("Product not found")
            new_stock = int(row["stock"]) + delta
            if new_stock < 0:
                cur.execute("ROLLBACK;")
                raise ValueError("Insufficient stock")
            cur.execute(
                "UPDATE products SET stock = ?, updated_at = ? WHERE id = ?",
                (new_stock, now_iso(), row["id"]),
            )
            cur.execute("COMMIT;")
```

### src/services/sales_service.py
- Sales service handles transactional checkout, stock decrements, and totals using SQLite transactions for integrity.[3]

```python
# src/services/sales_service.py
from __future__ import annotations

from typing import Iterable, List, Optional, Tuple
from src.models.database import Database
from src.utils.validators import validate_quantity, validate_sku
from src.utils.helpers import now_iso

Item = Tuple[str, int]  # (sku, quantity)

class SalesService:
    def __init__(self, db: Database) -> None:
        self.db = db

    def checkout(self, items: Iterable[Item], customer_id: Optional[int] = None) -> int:
        # items: iterable of (sku, qty)
        normalized: List[Item] = []
        for sku, qty in items:
            normalized.append((validate_sku(sku), validate_quantity(qty)))

        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            sale_ts = now_iso()
            cur.execute(
                "INSERT INTO sales (customer_id, total, created_at) VALUES (?, 0, ?)",
                (customer_id, sale_ts),
            )
            sale_id = int(cur.lastrowid)
            running_total = 0.0

            for sku, qty in normalized:
                prod = cur.execute(
                    "SELECT id, price, stock FROM products WHERE sku = ?",
                    (sku,),
                ).fetchone()
                if not prod:
                    cur.execute("ROLLBACK;")
                    raise LookupError(f"Product not found: {sku}")
                if prod["stock"] < qty:
                    cur.execute("ROLLBACK;")
                    raise ValueError(f"Insufficient stock for {sku}")

                new_stock = int(prod["stock"]) - qty
                unit_price = float(prod["price"])
                line_total = round(unit_price * qty, 2)
                running_total = round(running_total + line_total, 2)

                cur.execute(
                    "UPDATE products SET stock = ?, updated_at = ? WHERE id = ?",
                    (new_stock, sale_ts, prod["id"]),
                )
                cur.execute(
                    """
                    INSERT INTO sale_items (sale_id, product_id, quantity, unit_price, line_total)
                    VALUES (?, ?, ?, ?, ?)
                    """,
                    (sale_id, prod["id"], qty, unit_price, line_total),
                )

            cur.execute("UPDATE sales SET total = ? WHERE id = ?", (running_total, sale_id))
            cur.execute("COMMIT;")
            return sale_id
```

### src/services/report_service.py
- Reporting service provides daily summaries and top-selling products using GROUP BY queries on sales and line items.[3]

```python
# src/services/report_service.py
from __future__ import annotations

from typing import List, Tuple, Optional
from datetime import datetime, timedelta
from src.models.database import Database

class ReportService:
    def __init__(self, db: Database) -> None:
        self.db = db

    def daily_sales_total(self, date_iso: str) -> float:
        # date_iso: "YYYY-MM-DD"
        start = f"{date_iso}T00:00:00Z"
        end = f"{date_iso}T23:59:59Z"
        with self.db.get_connection() as conn:
            row = conn.execute(
                "SELECT COALESCE(SUM(total), 0) as total FROM sales WHERE created_at BETWEEN ? AND ?",
                (start, end),
            ).fetchone()
            return float(row["total"])

    def top_selling_products(self, limit: int = 5) -> List[Tuple[str, int, float]]:
        # returns list of (sku, quantity_sold, revenue)
        with self.db.get_connection() as conn:
            rows = conn.execute(
                """
                SELECT p.sku as sku,
                       SUM(si.quantity) as qty_sold,
                       ROUND(SUM(si.line_total), 2) as revenue
                FROM sale_items si
                JOIN products p ON p.id = si.product_id
                GROUP BY p.sku
                ORDER BY qty_sold DESC, revenue DESC
                LIMIT ?
                """,
                (int(limit),),
            ).fetchall()
            return [(r["sku"], int(r["qty_sold"]), float(r["revenue"])) for r in rows]
```

### src/cli/menu.py
- Interactive menu wraps services for a beginner-friendly flow and maps well to hands-on practice sessions.[4]

```python
# src/cli/menu.py
from __future__ import annotations

from src.models.database import Database
from src.services.inventory_service import InventoryService
from src.services.sales_service import SalesService
from src.services.report_service import ReportService
from src.utils.helpers import money

def run_menu() -> None:
    db = Database()
    db.initialize_schema()
    inv = InventoryService(db)
    sales = SalesService(db)
    rpt = ReportService(db)

    while True:
        print("\nInventory & Sales (POS)")
        print("1) List products")
        print("2) Add product")
        print("3) Update product")
        print("4) Remove product")
        print("5) Adjust stock")
        print("6) Checkout (sell)")
        print("7) Daily sales total")
        print("8) Top selling products")
        print("0) Exit")

        choice = input("Select: ").strip()
        try:
            if choice == "1":
                products = inv.list_products()
                print("\nSKU         Name                     Price   Stock")
                print("----------- ------------------------ ------- -----")
                for p in products:
                    print(f"{p.sku:<11} {p.name:<24} {money(p.price):>7} {p.stock:>5}")
            elif choice == "2":
                sku = input("SKU: ")
                name = input("Name: ")
                price = float(input("Price: "))
                stock = int(input("Initial stock: ") or "0")
                inv.add_product(sku, name, price, stock)
                print("Product added.")
            elif choice == "3":
                sku = input("SKU to update: ")
                name = input("New name (blank to skip): ") or None
                price_raw = input("New price (blank to skip): ")
                price = float(price_raw) if price_raw else None
                inv.update_product(sku, name=name, price=price)
                print("Product updated.")
            elif choice == "4":
                sku = input("SKU to remove: ")
                inv.remove_product(sku)
                print("Product removed.")
            elif choice == "5":
                sku = input("SKU to adjust: ")
                delta = int(input("Adjustment (e.g., +5 or -3): "))
                inv.adjust_stock(sku, delta)
                print("Stock adjusted.")
            elif choice == "6":
                print("Enter items as SKU:QTY,SKU2:QTY2 ...")
                raw = input("Items: ").strip()
                items = []
                if raw:
                    for token in raw.split(","):
                        sku, qty = token.split(":")
                        items.append((sku.strip(), int(qty)))
                sale_id = sales.checkout(items)
                print(f"Sale complete. ID={sale_id}")
            elif choice == "7":
                date_iso = input("Date (YYYY-MM-DD): ").strip()
                total = rpt.daily_sales_total(date_iso)
                print(f"Total sales on {date_iso}: {money(total)}")
            elif choice == "8":
                n = int(input("How many top products? ") or "5")
                rows = rpt.top_selling_products(n)
                print("\nSKU         Qty   Revenue")
                print("----------- ----- --------")
                for sku, qty, revenue in rows:
                    print(f"{sku:<11} {qty:>5} {money(revenue):>8}")
            elif choice == "0":
                print("Bye.")
                return
            else:
                print("Invalid selection.")
        except Exception as e:
            print(f"Error: {e}")
```

### src/cli/interface.py
- Argparse CLI with subcommands for products, sales, and reports aligns to common Python CLI practices and enables automation in scripts.[4]

```python
# src/cli/interface.py
from __future__ import annotations

import argparse
from typing import List, Tuple

from src.models.database import Database
from src.services.inventory_service import InventoryService
from src.services.sales_service import SalesService
from src.services.report_service import ReportService
from src.utils.helpers import money

def _parse_items(items_str: str) -> List[Tuple[str, int]]:
    items: List[Tuple[str, int]] = []
    if not items_str:
        return items
    for token in items_str.split(","):
        sku, qty = token.split(":")
        items.append((sku.strip(), int(qty)))
    return items

def cli_main(argv: list[str] | None = None) -> None:
    parser = argparse.ArgumentParser(prog="pos", description="Inventory & Sales (POS) CLI")
    sub = parser.add_subparsers(dest="cmd", required=True)

    # Products
    p_list = sub.add_parser("products:list", help="List products")
    p_add = sub.add_parser("products:add", help="Add a product")
    p_add.add_argument("--sku", required=True)
    p_add.add_argument("--name", required=True)
    p_add.add_argument("--price", required=True, type=float)
    p_add.add_argument("--stock", type=int, default=0)

    p_update = sub.add_parser("products:update", help="Update product")
    p_update.add_argument("--sku", required=True)
    p_update.add_argument("--name")
    p_update.add_argument("--price", type=float)

    p_remove = sub.add_parser("products:remove", help="Remove product")
    p_remove.add_argument("--sku", required=True)

    p_adjust = sub.add_parser("products:adjust", help="Adjust stock")
    p_adjust.add_argument("--sku", required=True)
    p_adjust.add_argument("--delta", required=True, type=int)

    # Sales
    s_checkout = sub.add_parser("sales:checkout", help="Checkout items")
    s_checkout.add_argument("--items", required=True, help="SKU:QTY,SKU2:QTY2")
    s_checkout.add_argument("--customer-id", type=int)

    # Reports
    r_daily = sub.add_parser("reports:daily", help="Daily sales total")
    r_daily.add_argument("--date", required=True, help="YYYY-MM-DD")

    r_top = sub.add_parser("reports:top-products", help="Top selling products")
    r_top.add_argument("--limit", type=int, default=5)

    args = parser.parse_args(argv)

    db = Database()
    db.initialize_schema()
    inv = InventoryService(db)
    sales = SalesService(db)
    rpt = ReportService(db)

    if args.cmd == "products:list":
        products = inv.list_products()
        for p in products:
            print(f"{p.sku}\t{p.name}\t{money(p.price)}\t{p.stock}")
    elif args.cmd == "products:add":
        inv.add_product(args.sku, args.name, args.price, args.stock)
        print("OK")
    elif args.cmd == "products:update":
        inv.update_product(args.sku, name=args.name, price=args.price)
        print("OK")
    elif args.cmd == "products:remove":
        inv.remove_product(args.sku)
        print("OK")
    elif args.cmd == "products:adjust":
        inv.adjust_stock(args.sku, args.delta)
        print("OK")
    elif args.cmd == "sales:checkout":
        items = _parse_items(args.items)
        sale_id = sales.checkout(items, customer_id=args.customer_id)
        print(f"SALE_ID={sale_id}")
    elif args.cmd == "reports:daily":
        total = rpt.daily_sales_total(args.date)
        print(f"{money(total)}")
    elif args.cmd == "reports:top-products":
        rows = rpt.top_selling_products(args.limit)
        for sku, qty, revenue in rows:
            print(f"{sku}\t{qty}\t{money(revenue)}")
```

### tests/test_models.py
- Model tests validate simple construction and row conversion to ensure schema compatibility at the edges.[3]

```python
# tests/test_models.py
from src.models.product import Product

def test_product_dataclass():
    p = Product(
        id=None,
        sku="ABC-123",
        name="Pen",
        price=12.5,
        stock=10,
        created_at="2024-01-01T00:00:00Z",
        updated_at="2024-01-01T00:00:00Z",
    )
    assert p.sku == "ABC-123"
    assert p.stock == 10
```

### tests/test_utils.py
- Utility tests check validators to ensure consistent input constraints at the boundary before hitting the database.[3]

```python
# tests/test_utils.py
import pytest
from src.utils.validators import validate_sku, validate_price, validate_quantity, validate_email

def test_validate_sku_ok():
    assert validate_sku("ab-12").startswith("AB-")

def test_validate_price_ok():
    assert validate_price(3.456) == 3.46

def test_validate_quantity_ok():
    assert validate_quantity(3) == 3

def test_validate_email_ok():
    assert validate_email("a@b.com") == "a@b.com"

def test_validate_email_none():
    assert validate_email(None) is None

def test_validate_price_negative():
    with pytest.raises(ValueError):
        validate_price(-1)
```

### tests/test_services.py
- Service tests run against a temporary SQLite file to validate product creation, listing, and a checkout reducing stock within a transaction.[3]

```python
# tests/test_services.py
from pathlib import Path
from src.models.database import Database
from src.services.inventory_service import InventoryService
from src.services.sales_service import SalesService

def test_inventory_and_sales(tmp_path: Path):
    db = Database(db_path=tmp_path / "test.db")
    db.initialize_schema()
    inv = InventoryService(db)
    sales = SalesService(db)

    inv.add_product("ABC-1", "Pen", 10.0, 5)
    inv.add_product("XYZ-2", "Notebook", 50.0, 3)
    products = inv.list_products()
    assert len(products) == 2

    sale_id = sales.checkout([("ABC-1", 2), ("XYZ-2", 1)])
    assert isinstance(sale_id, int)

    p1 = inv.get_product_by_sku("ABC-1")
    p2 = inv.get_product_by_sku("XYZ-2")
    assert p1.stock == 3
    assert p2.stock == 2
```

### requirements.txt
- Only pytest is required; everything else uses Python’s standard library to keep deployment simple and portable.[4]

```
pytest>=7.0
```

### README.md
- The README documents setup, commands, and how the project maps to Module 1 topics and exercises using src layout, sqlite3, and argparse.[1][4][3]

```markdown
# Inventory & Sales (POS)

A teaching-friendly, production-quality POS implemented in Python with a src layout, SQLite storage, and an argparse CLI.

## Quickstart
1) Create venv
   python -m venv .venv && . .venv/bin/activate  (Windows: .venv\Scripts\activate)

2) Install
   pip install -r requirements.txt

3) Initialize DB and run CLI
   python -m src.main
   # or use subcommands:
   python -m src.main products:add --sku ABC-1 --name "Pen" --price 10 --stock 5
   python -m src.main products:list
   python -m src.main sales:checkout --items "ABC-1:2"
   python -m src.main reports:daily --date 2025-09-16

## Commands
- products:list
- products:add --sku --name --price [--stock]
- products:update --sku [--name] [--price]
- products:remove --sku
- products:adjust --sku --delta
- sales:checkout --items "SKU:QTY,SKU2:QTY2" [--customer-id]
- reports:daily --date YYYY-MM-DD
- reports:top-products [--limit N]

## How this maps to Module 1
- Core Python syntax, variables, operators, and control flow: see services and CLI modules using types, conditionals, loops, and functions.
- Data structures: products, items, and reports use lists, tuples, and dictionaries.
- Functions and modules: code is organized into reusable modules with pure functions and services.
- OOP: dataclasses for Product, Customer, Sale/SaleItem encapsulate domain entities.
- Standard library modules: argparse for CLI, sqlite3 for DB, logging for diagnostics.
- File I/O: SQLite database file and structured project tree with src layout.
- SQL basics and advanced queries: schema creation, SELECTs with grouping, and transactional checkouts.

## Tests
Run pytest from the project root:
   pytest -q
```

### .gitignore
- Ignores virtual environments, bytecode, and miscellaneous editor files to keep the repository clean and reproducible.[2]

```
# Python
__pycache__/
*.pyc
*.pyo
*.pyd
*.pytest_cache/
*.coverage
.coverage*
htmlcov/

# Envs
.venv/
env/
venv/

# OS / Editors
.DS_Store
.idea/
.vscode/
*.swp

# Data
*.sqlite
*.db
!.gitkeep
```

### Running examples
- Example CLI flow: add two products, list them, sell items, then get daily totals and top products.[4]
- The database file is created automatically on first connection, and schema is initialized in main and CLI entry points for a smooth first-run experience.[3]

### Module 1 coverage notes
- The project exercises core Python, file layout, CLI, SQL DDL/DML, and Python-to-SQL integration, directly matching the listed weeks and hands-on tasks through real POS workflows and reporting.[1][4][3]

### Extending next
- Add receipt printing, discounts, taxes, or customer accounts, leveraging argparse subcommands and additional tables or fields as needed while keeping transactions atomic in sqlite3.[4][3]

### How to use both modes
- Run interactive menu with no arguments for guided exploration or pass subcommands for scripted operations, both powered by the standard library argparse and a shared service layer.[4]

### Notes on packaging later
- The src layout supports future packaging and distribution with pyproject.toml if desired, without changing imports or code structure.[1]
